## Steps to automate

* Get Text Corpus
* Remove punctuations
* Remove \n
* Make lower case
* tokenize
* lemmantize (choose method)
* Vectorize
* Remove stop words (try)
* TF-IDF Normalization
* transform, train and fit model (choose model)
* transform and predict test set
* Return prediction

In [2]:
from bs4 import BeautifulSoup

import re

import os

import nltk 
import contractions


import string
import pattern
from pattern.en import lemma, lexeme

from nltk import pos_tag
from textblob import TextBlob, Word

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [ ]:
kodaline_path = os.path.abspath('./Kodaline/') + '/'

In [ ]:
nwa_path = os.path.abspath('./NWA/') + '/'

In [ ]:
songlinks = [ kodaline_path + song for song in os.listdir('./Kodaline/')[:20]]

In [ ]:
nwa = [ nwa_path + song for song in os.listdir('./NWA/')[:20]]

In [ ]:
for song in nwa:
    songlinks.append(song)

In [ ]:
artist = ['Kodaline']*20 + ['NWA']*20

In [ ]:
lyrics_corpus = []
for song in songlinks:
    with open(song, 'r') as file:
        text = file.read()
        file.close()
    lyrics_corpus.append(text)

In [ ]:
lemm = WordNetLemmatizer()

In [ ]:
songtext = "".join([ch for ch in lyrics_corpus[0] if ch not in string.punctuation])

In [ ]:
songtext = re.sub('\n', ' ', songtext)

In [ ]:
songtext = songtext.lower()

In [ ]:
songtext

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.wn.ADJ,
                "N": wordnet.wn.NOUN,
                "V": wordnet.wn.VERB,
                "R": wordnet.wn.ADV}

    return tag_dict.get(tag, wordnet.wn.NOUN)

In [ ]:
lemme_nltk = " ".join([lemm.lemmatize(word, pos=get_wordnet_pos(word)) for word in songtext.split() ])

In [ ]:
sent = TextBlob(songtext)

In [ ]:
def lemmatize_with_postag(song):
    sent = TextBlob(song)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [ ]:
lemme_blob = lemmatize_with_postag(songtext)

In [ ]:
tokens = TreebankWordTokenizer()

In [ ]:
song_tokens = tokens.tokenize(songtext)

In [ ]:
len(song_tokens)

In [ ]:
lemme_pattern = " ".join([lemma(word) for word in songtext.split()])

In [ ]:
lemme_blob

In [ ]:
lemme_nltk

In [ ]:
lemme_pattern

In [ ]:
expanded_words = [contractions.fix(word) for word in song_fixed.split()]   

In [ ]:
expanded_text = ' '.join(expanded_words)
expanded_text

In [ ]:
song_fixed = expanded_text.replace(symbol, "'")

In [ ]:
char = [char for char in songtext if not (char.isalnum() | char.isspace())]

In [ ]:
char1 = [char for char in lyrics_corpus[3] if not (char.isalnum() | char.isspace())]

In [ ]:
songtext1 = "".join([ch for ch in songtext if (ch.isalnum() | ch.isspace())])
songtext1

In [ ]:
expanded_new = " ".join([contractions.fix(word) for word in songtext1.split()])
expanded_new

* Remove punctuations from song
* Rmove new line character
* expand contractions in song
* lemmantize song
* tokenize song words
* vectorize and remove stop words
* tfidf transformation
* classifier

In [ ]:
def remove_punc(corpus):
    new_corpus = []
    for song in corpus:
        song = "".join([ch for ch in song if (ch.isalnum() | ch.isspace())])
        new_corpus.append(song)
    return new_corpus

In [ ]:
new_corpus = remove_punc(lyrics_corpus)

In [ ]:
def remove_new_line(corpus):
    new_corpus = []
    for song in corpus:
        song = re.sub('\n', ' ', song)
        song = song.lower()
        new_corpus.append(song)
    return new_corpus

In [ ]:
new_corpus = remove_new_line(new_corpus)

In [ ]:
def expand_contraction(corpus):
    new_corpus = []
    for song in corpus:
        song = " ".join([contractions.fix(word) for word in song.split()])
        new_corpus.append(song)
    return new_corpus

In [ ]:
new_corpus = expand_contraction(new_corpus)

In [ ]:
def lemmatize_song(corpus, method='nltk'):
    new_corpus = []
    try:
        if method == 'nltk':
            for song in corpus:
                lemm = WordNetLemmatizer()
                song = " ".join([lemm.lemmatize(word, pos=get_wordnet_pos(word)) for word in song.split()])
                new_corpus.append(song)
        elif method == 'textblob':
            for song in corpus:
                song = lemmatize_with_postag(song)
                new_corpus.append(song)
        elif method == 'pattern':
            for song in corpus:
                song = " ".join([lemma(word) for word in song.split()])
                new_corpus.append(song)
        return new_corpus
        
    except:
        print(f"This is not an exceptable method. User nltk, textblob or pattern")

In [ ]:
nltk_corpus = lemmatize_song(corpus=new_corpus)

In [ ]:
textblob_corpus = lemmatize_song(corpus=new_corpus, method='textblob')

In [ ]:
pattern_corpus = lemmatize_song(corpus=new_corpus, method='pattern')

In [ ]:
cv = CountVectorizer(stop_words='english')

In [ ]:
vec_song = cv.fit_transform(pattern_corpus)

In [ ]:
vec_song

In [ ]:
tf = TfidfTransformer()

In [ ]:
vec_song_new = tf.fit_transform(vec_song)

In [ ]:
vec_song_new

In [ ]:
bayes_model = MultinomialNB()

In [ ]:
bayes_model.fit(vec_song_new, artist)

In [ ]:
bayes_model.score(vec_song_new, artist)

In [ ]:
bayes_model.predict_proba(vec_song_new)

In [ ]:
songlinks_test = [ kodaline_path + song for song in os.listdir('./Kodaline/')[21:]]

In [ ]:
nwa_test = [ nwa_path + song for song in os.listdir('./NWA/')[21:]]

In [ ]:
for song in nwa_test:
    songlinks_test.append(song)

In [ ]:
test_corpus = []
for song in songlinks:
    with open(song, 'r') as file:
        text = file.read()
        file.close()
    test_corpus.append(text)

In [ ]:
test_corpus = remove_punc(test_corpus)

In [ ]:
test_corpus = remove_new_line(test_corpus)

In [ ]:
test_corpus = expand_contraction(test_corpus)

In [ ]:
test_corpus = lemmatize_song(test_corpus, method='pattern')

In [ ]:
vec_test = cv.transform(test_corpus)
vec_test

In [ ]:
vec_test_new = tf.transform(vec_test)
vec_test_new

In [ ]:
bayes_model.predict(vec_test_new)

In [ ]:
bayes_model.predict_proba(vec_test_new)

In [ ]:
log_model = LogisticRegression()

In [ ]:
log_model.fit(vec_song_new, artist)

In [ ]:
log_model.score(vec_song_new, artist)

In [ ]:
log_model.predict(vec_test_new)